A way to determine areas of greatest precipitation and generate imagery for that sector.

In [ ]:
from awips.dataaccess import DataAccessLayer
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from metpy.units import units
import numpy as np
from shapely.geometry import Point, Polygon

%matplotlib inline

conus=[-120, -65, 28, 50]
conus_envelope = Polygon([(conus[0],conus[2]),(conus[0],conus[3]),
                          (conus[1],conus[3]),(conus[1],conus[2]),
                          (conus[0],conus[2])])

DataAccessLayer.changeEDEXHost("edex-cloud.unidata.ucar.edu")
request = DataAccessLayer.newDataRequest("grid", envelope=conus_envelope)
request.setLocationNames("NAM40")
request.setLevels("0.0SFC")
request.setParameters("TP")

cycles = DataAccessLayer.getAvailableTimes(request, True)
times = DataAccessLayer.getAvailableTimes(request)
fcstRun = DataAccessLayer.getForecastRun(cycles[-2], times)

Calculate accumulated precipitation

In [ ]:
for i, tt in enumerate(fcstRun):
    response = DataAccessLayer.getGridData(request, [tt])
    grid = response[0]
    if i>0:
        data += grid.getRawData()
    else:
        data = grid.getRawData()
    data[data <= -9999] = 0
    print(data.min(), data.max(), grid.getDataTime().getFcstTime()/3600)
    

lons, lats = grid.getLatLonCoords()
bbox = [lons.min(), lons.max(), lats.min(), lats.max()]
fcstHr = int(grid.getDataTime().getFcstTime()/3600)

tp_inch = data * (0.0393701)
print(tp_inch.min(), tp_inch.max())

Determine lat/lon of maximum rainfall value:

In [ ]:
ii,jj = np.where(tp_inch==tp_inch.max())
i=ii[0]
j=jj[0]
point = Point(lons[i][j], lats[i][j])

Draw CONUS map

In [ ]:
def make_map(bbox, projection=ccrs.PlateCarree()):
    fig, ax = plt.subplots(figsize=(20, 14),
            subplot_kw=dict(projection=projection))
    ax.set_extent(bbox)
    ax.coastlines(resolution='50m')
    return fig, ax

cmap = plt.get_cmap('rainbow')
fig, ax = make_map(bbox=bbox)
cs = ax.pcolormesh(lons, lats, tp_inch, cmap=cmap)
cbar = fig.colorbar(cs, shrink=0.7, orientation='horizontal')
cbar.set_label(grid.getLocationName() + " Total precipitation in inches, " \
               + str(fcstHr) + "-hr fcst valid " + str(grid.getDataTime().getRefTime()))

ax.scatter(point.x, point.y, s=300,
           transform=ccrs.PlateCarree(),marker="+",facecolor='black')

inc = 3.5
box=[point.x-inc, point.x+inc, point.y-inc, point.y+inc]
polygon = Polygon([(box[0],box[2]),(box[0],box[3]), 
                   (box[1],box[3]),(box[1],box[2]),
                   (box[0],box[2])])
ax.add_geometries([polygon], ccrs.PlateCarree(), facecolor='none', edgecolor='white', linewidth=2)

Now create a new gridded data request with a geometry envelope for our Region of Interest

In [ ]:
request = DataAccessLayer.newDataRequest("grid", envelope=polygon)
request.setLocationNames("HRRR")
request.setLevels("0.0SFC")
request.setParameters("TP")

cycles = DataAccessLayer.getAvailableTimes(request, True)
times = DataAccessLayer.getAvailableTimes(request)
fcstRun = DataAccessLayer.getForecastRun(cycles[-2], times)


for i, tt in enumerate(fcstRun):
    response = DataAccessLayer.getGridData(request, [tt])
    grid = response[0]
    if i>0:
        data += grid.getRawData()
    else:
        data = grid.getRawData()
    data[data <= -9999] = 0
    print(data.min(), data.max(), grid.getDataTime().getFcstTime()/3600)
    

lons, lats = grid.getLatLonCoords()
bbox = [lons.min(), lons.max(), lats.min(), lats.max()]
fcstHr = int(grid.getDataTime().getFcstTime()/3600)

tp_inch = data * (0.0393701)
print(tp_inch.min(), tp_inch.max())

def make_map(bbox, projection=ccrs.PlateCarree()):
    fig, ax = plt.subplots(figsize=(20, 14),
            subplot_kw=dict(projection=projection))
    ax.set_extent(bbox)
    ax.coastlines(resolution='50m')
    return fig, ax

cmap = plt.get_cmap('rainbow')
fig, ax = make_map(bbox=box)
cs = ax.pcolormesh(lons, lats, tp_inch, cmap=cmap)
cbar = fig.colorbar(cs, shrink=0.7, orientation='horizontal')
cbar.set_label(grid.getLocationName() + " Total precipitation in inches, " \
               + str(fcstHr) + "-hr fcst valid " + str(grid.getDataTime().getRefTime()))